# Introduction
This is a demo of dbt work flow using an data ingestion as an example. In finance data ETL (Extract, Transform, Load)
important to have sanitity check of input data, to avoid the ingesting invalid data....

In [1]:
import pandas as pd
pd.read_csv("/usr/app/demo/dbt_spark_demo_prj/seeds/transactions/raw__transactions.csv")

,id,amount,currency,status
0,1,100,USD,ACTIVE
1,2,200,SGD,ACTIVE
2,3,150,EUR,ACTIVE
3,4,120,USD,INACTIVE
4,5,180,SGD,INACTIVE
5,6,300,JPY,ACTIVE


## Project file structure

In [2]:
!tree -L 2 --sort="name" /usr/app | grep -v ".*\.toml$" | grep -v ".*\.md$" | grep -v ".*files$"

/usr/app
├── dbt
│   ├── dbt.Dockerfile
│   ├── dbt_spark_demo_prj
│   ├── docker-compose.yml
│   └── requirements.txt
├── dbt-spark
│   ├── dagger
│   ├── dbt
│   ├── docker
│   ├── docker-compose.yml
│   ├── scripts
│   ├── test.env.example
│   └── tests
├── demo
│   ├── dbt-spark-demo.html
│   ├── dbt-spark-demo.ipynb
│   ├── dbt-spark-demo_files
│   ├── dbt_spark_demo_prj
│   ├── generate-documentation.sh
│   └── profiles.yml
└── docker-common.sh



# Environment setup

## Docker
### Shared network for docker containers
```bash
source ./docker-common.sh
```

## Install jupyter notebook for running this demo in notebook 
This notebook is executed using jupyer-notebook kernel (http://127.0.0.1:8888) of the loaded custom-dbt container. The container is loaded using command below
```bash
jupyter notebook --NotebookApp.token='' --NotebookApp.password='' --NotebookApp.disable_check_xsrf=True --allow-root --ip=0.0.0.0 --port=8888 --no-browser
```
use allow-root for demo purpose only, not for production due to cyber security


In [3]:
%%bash
export TZ='Asia/Singapore'
date
which python
pwd

Sun Jun 29 23:09:16 +08 2025
/usr/local/bin/python
/usr/app/demo


### Connection to read table in Hive meta-data store

## Variables

In [4]:
from dotenv import load_dotenv
import os
!pwd

!cd ../
!pwd

load_dotenv()
spark_thrift_service_name=os.getenv('SPARK_CONTAINER_SERVICE_NAME')
dbt_project_dir=os.getenv('DBT_PROJECT_DIR')
dbt_project_name=os.getenv('DBT_PROJECT_NAME')
print(spark_thrift_service_name)
print(dbt_project_dir)
print(dbt_project_name)

/usr/app/demo
/usr/app/demo
dbt-spark3-thrift
/usr/app/dbt/dbt_spark_demo_prj
dbt_spark_demo_prj


## Helper codes

In [5]:
from pyhive import hive
import pandas as pd
from sqlalchemy.engine import create_engine

def check_table(host=spark_thrift_service_name, message_to_be_printed=""):
    # with hive.Connection(host=spark_thrift_service_name, port=10000, 
    #                     #    username='', 
    #                     database='default') as conn:

    #     # Query all tables in the default database
    #     df = pd.read_sql("SHOW TABLES", conn)

    #     # Display tables
    #     display(df)
    print(message_to_be_printed)
    engine = create_engine(f'hive://{spark_thrift_service_name}:10000/default')
    df = pd.read_sql("SHOW TABLES;", engine)
    # display(df)
    display(df)

def show_table_details(table_name, database_name='default',
                        # host=spark_thrift_service_name,
                        port=10000):
    # sqlAlchemy api fail here due to "sqlAlchemy relection": TODO investigate root cause
    # engine = create_engine(f'hive://{spark_thrift_service_name}:10000/default')
    # df = pd.read_sql(f"select * from {table_name} limit 10", engine)

    with hive.Connection(host=spark_thrift_service_name, port=port, database=database_name) as conn:
        df = pd.read_sql(f"select * from {database_name}.{table_name}", conn)

        display(df)

    
check_table()
print("------------")
# show_table_details("transactions")


,namespace,tableName,isTemporary
0,default,raw__transactions,False
1,default,transactions,False


------------


## DBT 

In [6]:
!dbt --version

Core:
  - installed: 1.10.2
  - latest:    1.10.2 - Up to date!

Plugins:
  - spark: 1.9.2 - Up to date!




## Verify DBT setup

### profiles.yml
```yml
formal_verification_prj_name:
  outputs:
    spark:
      type: spark
      method: thrift
      host: dbt-spark3-thrift
      schema: default
      connect_timeout: 30
      authentication: NONE
  target: spark

```

In [7]:
!cat ~/.dbt/profiles.yml

send_anonymous_usage_stats: false

dbt_spark_demo_prj:
  outputs:
    spark:
      type: spark
      method: thrift
      host: dbt-spark3-thrift
      port: 10000
      schema: default
      connect_timeout: 30
      authentication: NONE
  target: spark


### dbt_project.yml
```yml
name: 'formal_verification_prj_name'
version: '1.0.0'

# This setting configures which "profile" dbt uses for this project.
profile: 'formal_verification_prj_name'

# These configurations specify where dbt should look for different types of files.
# The `model-paths` config, for example, states that models in this project can be
# found in the "models/" directory. You probably won't need to change these!
model-paths: ["models"]
analysis-paths: ["analyses"]
test-paths: ["tests"]
seed-paths: ["seeds"]
macro-paths: ["macros"]
snapshot-paths: ["snapshots"]

clean-targets:         # directories to be removed by `dbt clean`
  - "target"
  - "dbt_packages"

models:
  formal_verification_prj_name:
    # +write_json: false
    +materialized: incremental
    +file_format: parquet    

```

In [8]:
!cat /usr/app/dbt/dbt_spark_demo_prj/dbt_project.yml


# Name your project! Project names should contain only lowercase characters
# and underscores. A good package name should reflect your organization's
# name or the intended use of these models
name: 'dbt_spark_demo_prj'
version: '1.0.0'

# This setting configures which "profile" dbt uses for this project.
profile: 'dbt_spark_demo_prj'

# These configurations specify where dbt should look for different types of files.
# The `model-paths` config, for example, states that models in this project can be
# found in the "models/" directory. You probably won't need to change these!
model-paths: ["models"]
analysis-paths: ["analyses"]
test-paths: ["tests"]
seed-paths: ["seeds"]
macro-paths: ["macros"]
snapshot-paths: ["snapshots"]

clean-targets:         # directories to be removed by `dbt clean`
  - "target"
  - "dbt_packages"


# Configuring models
# Full documentation: https://docs.getdbt.com/docs/configuring-models

# In this example config, we tell dbt to build all models in the example/


### dbt debug

In [9]:
!dbt debug --project-dir /usr/app/dbt/dbt_spark_demo_prj

#to print detailed messages for debugging 
#!dbt debug --debug --project-dir /usr/app/dbt/dbt_spark_demo_prj

15:09:26  Running with dbt=1.10.2
15:09:26  dbt version: 1.10.2
15:09:26  python version: 3.11.2
15:09:26  python path: /usr/local/bin/python3
15:09:26  os info: Linux-5.15.167.4-microsoft-standard-WSL2-x86_64-with-glibc2.31
15:09:26  Using profiles dir at /usr/app/demo
15:09:26  Using profiles.yml file at /usr/app/demo/profiles.yml
15:09:26  Using dbt_project.yml file at /usr/app/dbt/dbt_spark_demo_prj/dbt_project.yml
15:09:26  adapter type: spark
15:09:26  adapter version: 1.9.2
15:09:26  Configuration:
15:09:26    profiles.yml file [OK found and valid]
15:09:26    dbt_project.yml file [OK found and valid]
15:09:26  Required dependencies:
15:09:26   - git [OK found]

15:09:26  Connection:
15:09:26    host: dbt-spark3-thrift
15:09:26    port: 10000
15:09:26    cluster: None
15:09:26    endpoint: None
15:09:26    schema: default
15:09:26    organization: 0
15:09:26  Registered adapter: spark=1.9.2
15:09:26    Connection test: [OK connection ok]

15:09:26  All checks passed!


## dbt command comparisons
| Command           | Purpose                                  | What it does                                                             | Output                                    |
| ----------------- | ---------------------------------------- | ------------------------------------------------------------------------ | ----------------------------------------- |
| **`dbt compile`** | Prepares SQL code for execution          | Converts Jinja + macros into raw SQL; writes compiled files to `target/` | Compiled SQL files (no warehouse changes) |
| **`dbt run`**     | Builds models (tables/views)             | Executes compiled SQL to materialize models in the data warehouse        | Tables/views in the warehouse             |
| **`dbt test`**    | Validates data quality assumptions       | Runs data tests (e.g., `not_null`, `unique`, custom) on warehouse data   | Test pass/fail results                    |
| **`dbt seed`**    | Loads static CSV data into the warehouse | Uploads CSV files from the `seeds/` directory into tables                 | Tables containing seed data               |


# Prepare SQL

## Read dbt input csv 

* read seeds/transactions.csv, 
* dbt generate raw__transaction.csv
* dbt create table default.raw_transaction at the database 

### Verify database before the operation

In [10]:
!dbt run-operation drop_table --args '{"table_name": "default.raw__transactions"}'  
!dbt run-operation drop_table --args '{"table_name": "default.transactions"}' 

15:09:31  Running with dbt=1.10.2
15:09:31  Registered adapter: spark=1.9.2
15:09:32  Found 1 model, 1 test, 1 seed, 476 macros, 3 unit tests
15:09:37  Running with dbt=1.10.2
15:09:37  Registered adapter: spark=1.9.2
15:09:38  Found 1 model, 1 test, 1 seed, 476 macros, 3 unit tests


In [11]:
check_table(message_to_be_printed="Database table before the operation")
!dbt seed --log-level info --project-dir $dbt_project_dir
check_table(message_to_be_printed="\nDatabase table after the operation")

Database table before the operation


,namespace,tableName,isTemporary


15:09:43  Running with dbt=1.10.2
15:09:44  Registered adapter: spark=1.9.2
15:09:44  Found 1 model, 1 test, 1 seed, 476 macros, 3 unit tests
15:09:44  
15:09:44  Concurrency: 1 threads (target='spark')
15:09:44  
15:09:44  1 of 1 START seed file default.raw__transactions ............................... [RUN]
15:09:45  1 of 1 OK loaded seed file default.raw__transactions ........................... [INSERT 6 in 0.70s]
15:09:45  
15:09:45  Finished running 1 seed in 0 hours 0 minutes and 0.98 seconds (0.98s).
15:09:45  
15:09:45  Completed successfully
15:09:45  
15:09:45  Done. PASS=1 WARN=0 ERROR=0 SKIP=0 NO-OP=0 TOTAL=1

Database table after the operation


,namespace,tableName,isTemporary
0,default,raw__transactions,False


## dbt compile

#### removed target/compiled/

In [12]:
!rm -rf dbt_spark_demo_prj/target/compiled/
!ls dbt_spark_demo_prj/target/compile

ls: cannot access 'dbt_spark_demo_prj/target/compile': No such file or directory


In [13]:
!ls $dbt_project_dir

dbt_project.yml  logs  macros  models  seeds  target  tests


In [14]:
!dbt compile --log-level info --project-dir $dbt_project_dir

15:09:52  Running with dbt=1.10.2
15:09:52  Registered adapter: spark=1.9.2
15:09:53  Found 1 model, 1 test, 1 seed, 476 macros, 3 unit tests
15:09:53  
15:09:53  Concurrency: 1 threads (target='spark')
15:09:53  


In [15]:
!ls /usr/app/dbt/dbt_spark_demo_prj/target/compiled/dbt_spark_demo_prj

models	tests


## Build your models (dbt run)

## Delete database table prior dbt run that will create the table

In [16]:
check_table(message_to_be_printed="before")
!dbt run-operation drop_table --args '{"table_name": "default.transactions"}'
check_table(message_to_be_printed="\nafter")

before


,namespace,tableName,isTemporary
0,default,raw__transactions,False


15:09:59  Running with dbt=1.10.2
15:09:59  Registered adapter: spark=1.9.2
15:10:00  Found 1 model, 1 test, 1 seed, 476 macros, 3 unit tests

after


,namespace,tableName,isTemporary
0,default,raw__transactions,False


In [ ]:
!rm -r $dbt_project_dir/target/run
!dbt run-operation drop_table --args '{"table_name": "default.transactions"}' 
check_table(message_to_be_printed="before")

!dbt run --log-level info --project-dir $dbt_project_dir

print("\nAfter")
!tree $dbt_project_dir/target/run
check_table(message_to_be_printed="after")

15:10:05  Running with dbt=1.10.2
15:10:06  Registered adapter: spark=1.9.2


## test model using dbt

#### test name and its test vectors

In [ ]:
!cat dbt_spark_demo_prj/tests/unit/test_transactions.yml

#### run test

In [ ]:
%%bash
export DBT_PROJECT_DIR=/usr/app/dbt/dbt_spark_demo_prj
dbt test --log-level info

In [ ]:
!export TZ='Asia/Singapore'; date

In [ ]:
!chmod -R a+rw /usr/app

In [ ]:
assert False, "Stop execution here!"

# Apendix

## check the model built 

### Check the model built using Pyspark (In progress)

In [ ]:
!pip install pyspark==3.3.2

In [ ]:
%%bash
# For Debian/Ubuntu-based containers:
apt-get update && apt-get install -y openjdk-11-jdk

# For Alpine-based containers:
apk add openjdk11

In [ ]:
!ls /usr/lib/jvm/java-11-openjdk-amd64

In [ ]:
# !. /usr/spark/bin/load-spark-env.sh

# Typically in Debian/Ubuntu:
!export JAVA_HOME=$(dirname $(dirname $(readlink -f $(which java))))

# Or set it explicitly:
!export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("dbt_clean") \
    .config("spark.hadoop.hive.metastore.schema.verification", "true") \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()

# spark = SparkSession.builder \
#     .config("hive.metastore.uris", "thrift://dbt-hive-metastore:10000") \
#     .config("spark.sql.catalogImplementation", "hive") \
#     .enableHiveSupport() \
#     .getOrCreate()

spark.sql("USE default")  # Or your specific database
spark.sql("SHOW DATABASES").show()
# spark.sql("select * from transactions").show()
# # df.toPandas()

In [ ]:
spark.sql("SHOW TABLES IN default").show(truncate=False)

In [ ]:
# Query metastore system tables
spark.sql("""
  SELECT TBL_NAME, TBL_TYPE 
  FROM default.TBLS 
  JOIN default.DBS ON TBLS.DB_ID = DBS.DB_ID 
  WHERE DBS.NAME = 'default'
""").show(truncate=False)

In [ ]:
spark.conf.get("hive.metastore.uris")  # Should return your URI

In [ ]:
spark.conf.get("spark.sql.catalogImplementation")  # Should return "hive"

In [ ]:
from pyspark.sql import SparkSession

spark.stop()
spark = SparkSession.builder \
    .appName("DBTIntegration") \
    .config("hive.metastore.uris", "thrift://dbt-hive-metastore:9083") \
    .config("spark.sql.catalogImplementation", "hive") \
    .config("javax.jdo.option.ConnectionUserName", "dbt") \
    .config("javax.jdo.option.ConnectionPassword", "dbt") \
    .enableHiveSupport() \
    .getOrCreate()
spark.sql("SHOW TABLES IN default")

In [ ]:
!pip install pyspark==3.3.2

In [ ]:
# from pyhive import hive
# conn = hive.connect(host=spark_thrift_service_name, port=10000)
# cursor = conn.cursor()
# cursor.execute("SELECT * FROM default.transactions LIMIT 1")
# print(cursor.fetchall())

### Test the built model (using the test model) 

In [ ]:
### dbt test

In [ ]:
!dbt test --log-level info

In [ ]:
!cat /home/vin/01-prj/stripe/sql-formal-verification/formal_verification_prj_name/tests/unit/test_transactions.yml

In [ ]:
# %%bash
# cd /usr/app/dbt
# dbt compile --profiles-dir /usr/app/dbt  --project-dir /usr/app/dbt/dbt_spark_demo_prj
# # dbt run-operation drop_table --args '{"table_name": "default.transactions"}'
# dbt run-operation drop_view --args '{"view_name": "default.my_first_dbt_model"}' --profiles-dir /usr/app/dbt  --project-dir /usr/app/dbt/dbt_spark_demo_prj
# dbt run-operation drop_view --args '{"view_name": "default.my_second_dbt_model"}' --profiles-dir /usr/app/dbt  --project-dir /usr/app/dbt/dbt_spark_demo_prj

# Todo
## Audit compliance 
### time tracing with time-stamping
1. Add timestamp for loaded data using dbt model
```jinja
SELECT
    *,
    '{{ run_started_at }}'::timestamp AS added_at
FROM {{ ref('my_seed_table') }}
```
### File tracing with source and targets file path, tablename
